In [1]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import joblib

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
sys.path.append('/Users/hb26404/Desktop/内部模型/0建模脚本')
import myfuncs as mf
importlib.reload(mf)

(CVXPY) Jun 25 11:09:35 AM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 25 11:09:35 AM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.9.3963). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


<module 'myfuncs' from '/Users/hb26404/Desktop/内部模型/0建模脚本/myfuncs.py'>

In [3]:
left_fes = ['翡翠92',
 '明鉴92',
 '玲珑92',
 '琳琅93',
 'SC_C11010',
 '小满分V12PRO-互金版高定价2',
 '小满分消金定制版3',
 'Score_hk52',
 'Score_hy51',
 'Score_o34',
 'afc_yh_cus_dq_v2',
 'hello_v6',
 'ant_anti_insight_score_v6',
 '融安云联分-标准款',
 'yaosi_ty_2',
 'tuyuan_207',
 'jm_s1',
 '50002',
 'pudao_tengxun_external_dp_loan_score_v1',
 'pudao_tengxun_external_model_risk_v8_stongyong_score',
 'pudao_tengxun_external_dp_credit_score_v1',
 'youmeng_external_risk_score_y7_v1',
 'youmeng_external_risk_score_y7_v2',
 'youmeng_external_risk_score_y7_v3',
 'yidun_external_subscore1_v1',
 'yidun_external_subscore2_v1',
 'yidun_external_subscore3_v1',
 'yidun_external_subscore4_v1',
 'yidun_external_subscore5_v1',
 'yidun_external_subscore6_v1',
 'yidun_external_subscore7_v1',
 'yidun_external_subscore8_v1',
 'yidun_external_subscore9_v1',
 ]



left_fes_name= {
    
    # 信易搜相关
    "翡翠92": "xys_fc92", 
    "明鉴92": "xys_mj92", 
    "玲珑92": "xys_linglong92", 
    "琳琅93": "xys_linlang93", 
    # 电话邦相关
     "SC_C11010":'dhb_SC_C11010',
     
   #度小满
    
    "小满分V12PRO-互金版高定价2":'dxm_V12PRO-hjgdj2',
     "小满分消金定制版3":'dxm_xjdz3',
    
    # 恒指普惠相关
    "Score_hk52":'hengzhi_Score_hk52', 
    "Score_hy51":'hengzhi_Score_hy51',
    "Score_o34":'hengzhi_Score_o34',
   
    
    # 蚂蚁相关
    "afc_yh_cus_dq_v2":'mayi_afc_yh_cus_dq_v2', 
    "hello_v6":'mayi_hello_v6',
    "ant_anti_insight_score_v6":'ant_anti_insight_score_v6',
    
    # 百融相关
     "融安云联分-标准款":'bairong_yunlianfen',

     #避雷针
    "yaosi_ty_2":'blz_yaosi_ty_2', 
    'tuyuan_207':'blz_tuyuan_207',
    
    #极光
    "jm_s1":'jg_jm_s1', 

    # 火山相关
    "50002":'zj_50002',

    #腾讯
    "pudao_tengxun_external_dp_loan_score_v1":'pudao_tengxun_external_dp_loan_score_v1',
    "pudao_tengxun_external_model_risk_v8_stongyong_score":'pudao_tengxun_external_model_risk_v8_stongyong_score',
     "pudao_tengxun_external_dp_credit_score_v1":'pudao_tengxun_external_dp_credit_score_v1',

     #友盟
    "youmeng_external_risk_score_y7_v1":'youmeng_external_risk_score_y7_v1',
    "youmeng_external_risk_score_y7_v2":'youmeng_external_risk_score_y7_v2',
    "youmeng_external_risk_score_y7_v3":'youmeng_external_risk_score_y7_v3',
 
    
     #蚂蚁9个子分
       "yidun_external_subscore1_v1":'yidun_external_subscore1_v1',
       "yidun_external_subscore2_v1":'yidun_external_subscore2_v1',
       "yidun_external_subscore3_v1":'yidun_external_subscore3_v1',
       "yidun_external_subscore4_v1":'yidun_external_subscore4_v1',
       "yidun_external_subscore5_v1":'yidun_external_subscore5_v1',
       "yidun_external_subscore6_v1":'yidun_external_subscore6_v1',
       "yidun_external_subscore7_v1":'yidun_external_subscore7_v1',
       "yidun_external_subscore8_v1":'yidun_external_subscore8_v1',
       "yidun_external_subscore9_v1":'yidun_external_subscore9_v1',
       
}

fes_all = list(left_fes_name.values())

len(left_fes_name)

33

In [4]:
df_score = pd.read_csv('/Users/hb26404/Desktop/内部模型/c924项目A卡模型2505/2数据分析/merged_dfv3.csv')
# df_score['date'] = pd.to_datetime(df_score['apply_date'].astype('str')).astype('str')
# df_score['mon'] = df_score['date'].str[:7]
df_score = df_score[['mob_md5', 'apply_date']+left_fes]
df_score = df_score.rename(columns=left_fes_name)
df_score.shape

(300000, 35)

In [5]:
df_sam = pd.read_csv('/Users/hb26404/Desktop/内部模型/c924项目A卡模型2505/1样本及数据/外发样本/30W外发样本所有基本信息250617.csv')
df_sam.shape

(300000, 35)

In [6]:
df_merge = df_sam.merge(df_score, how='inner', on=['mob_md5', 'apply_date'])
df_merge.shape

(300000, 68)

In [7]:
df_merge[fes_all] = df_merge[fes_all].replace([-999, -2, -1], [np.nan, np.nan, np.nan])
df_merge['mon'] = df_merge['date'].str[:7] 
df_merge.shape

(300000, 69)

In [8]:
bins_dict = {'10分箱':10, '20分箱':20, '40分箱':40, '50分箱':50}
self_cut_dict = {}
for bins_one in bins_dict.keys():
    gap_tmp = np.round((900-300)/bins_dict[bins_one], 1)
    self_cut = list(np.arange(300, 900, gap_tmp))
    self_cut.remove(300)
    self_cut = [-np.inf] + self_cut + [np.inf]
    self_cut_dict[bins_one] = self_cut


In [10]:
model = joblib.load('/Users/hb26404/Desktop/内部模型/c924项目A卡模型2505/3模型训练/update3_lgbm_uplevel_v1_0.pkl')
model_fes = model.feature_name()


special_list = []
cat_fes = []
label_list = [  'label_fpd_7', 'label_fpd_15', 'label_fpd_30', 'label_2pd_7', 'label_2pd_15',
       'label_2pd_30', 'label_3pd_7', 'label_3pd_15', 'label_3pd_30',
       'label_4pd_15', 'label_4pd_30', 'label_5pd_30', 'label_6pd_30',]
y_label = 'label_2pd_15'
model_pred='model_pred'
model_score = 'model_score'
parts_col = 'mon'
date_col = 'date'

cut_date = '2024-08-01'
oot_date = '2025-03-15'

cut_date1 = '2024-10-01'

id_col = 'user_guid'

if_sort = False

model_scores = [model_score]

len(model_fes)


11

In [11]:
model_fes

['xys_fc92',
 'dxm_V12PRO-hjgdj2',
 'pudao_tengxun_external_dp_loan_score_v1',
 'feat_loan_risk_province_from_idcard',
 'blz_yaosi_ty_2',
 'zj_50002',
 'feat_loan_risk_gender_from_idcard',
 'mayi_afc_yh_cus_dq_v2',
 'feat_loan_risk_age_from_idcard',
 'mayi_hello_v6',
 'pudao_tengxun_external_dp_credit_score_v1']

In [74]:
def Prob2Score(prob, basePoint=500, PDO=200):
    # ll = np.polyfit([153,1083],[301,899],1)
    ll = [0.6430107, 202.6193548]
    score = int(basePoint - PDO /np.log(2) * np.log(prob/(1 - prob)))
    score = int(score*ll[0]+ll[1])
    score = min(max([score,300]),900)
    return score

df_merge[model_pred] = model.predict(df_merge[model_fes])
df_merge[model_score] = df_merge[model_pred].apply(lambda x: Prob2Score(x,))


In [75]:
# df_out = df_merge[['user_guid', 'date', 'model_pred', 'model_score']+model_fes].sample(1000)
# df_out.shape

In [76]:
df_tt = df_merge[(df_merge.date<=oot_date)&(df_merge[y_label].isin([0,1]))]
train, test=train_test_split(df_tt, test_size=0.2, stratify=df_tt[y_label], random_state=1234)

df_oot = df_merge[(df_merge.date>oot_date)&(df_merge[y_label].isin([0,1]))]
df_oot1 = df_merge[(df_merge.date>oot_date)&(df_merge[y_label].notnull())]
# df_oot = df_merge[(df_merge.date>=cut_date)& (df_merge.date<cut_date1) &(df_merge[y_label].isin([0,1]))]
# df_oot1 = df_merge[(df_merge.date>=cut_date)& (df_merge.date<cut_date1)&(df_merge[y_label].notnull())]
df_oot1[y_label] = df_oot1[y_label].replace(0.5,0)

samples_info = pd.DataFrame(index=['train','test', 'oot(去除灰)', 'oot(灰为好)'])
samples_info['样本量'] = [train[y_label].count(),test[y_label].count(),df_oot[y_label].count(),df_oot1[y_label].count(),]
samples_info['样本坏量'] = [train[y_label].sum(),test[y_label].sum(),df_oot[y_label].sum(),df_oot1[y_label].sum()]
samples_info['样本坏率'] = [train[y_label].mean(),test[y_label].mean(),df_oot[y_label].mean(),df_oot1[y_label].mean()]
samples_info

,样本量,样本坏量,样本坏率
train,161799,5106.0,0.031558
test,40450,1277.0,0.031570
oot(去除灰),23843,973.0,0.040809
oot(灰为好),27814,973.0,0.034982


In [15]:
cat_fes = ['feat_loan_risk_province_from_idcard']

In [16]:
#计算特征信息
psi_oot = df_oot
psi_train = train

psi_df = pd.DataFrame()
psi_to = {}
iv_res = {}
iv_bins = {}

train_lossrate = {}
oot_lossrate = {}
fes_optb = {}

for col in tqdm(model_fes):
    base_col = psi_train[col]
    oot_col = psi_oot[col]

    # if col in cat_fes:
        
    #     psi_to[col],_ , _= mf.calculate_psi(base_col, oot_col, bins=5, min_sample=10, numerical=False)
    # else:
        
    psi_to[col],_, _ = mf.calculate_psi(base_col, oot_col, bins=5, min_sample=10, special_list=special_list, numerical=True)
        
    if col in cat_fes:
        iv_res[col], iv_bins[col], fes_optb[col] = mf.calculate_iv(base_col, psi_train[y_label], name=col, max_n_bins=5, special_codes=special_list, min_prebin_size=0.05, split_digits=6, dtype='categorical')
    else:
        iv_res[col], iv_bins[col], fes_optb[col] = mf.calculate_iv(base_col, psi_train[y_label], name=col, max_n_bins=5, special_codes=special_list, min_prebin_size=0.05, split_digits=6, dtype='numerical')

    train_lossrate[col] = psi_train[col].isnull().mean()
    oot_lossrate[col] = psi_oot[col].isnull().mean()



psi_df['to'] = pd.Series(psi_to)

# psi_df['to2'] = pd.Series(psi_to2)
iv_res = pd.Series(iv_res)

100%|██████████| 11/11 [00:01<00:00, 10.50it/s]


In [17]:
psi_df.max()

to    0.024481
dtype: float64

In [18]:
list(np.sort(model_fes))

['blz_yaosi_ty_2',
 'dxm_V12PRO-hjgdj2',
 'feat_loan_risk_age_from_idcard',
 'feat_loan_risk_gender_from_idcard',
 'feat_loan_risk_province_from_idcard',
 'mayi_afc_yh_cus_dq_v2',
 'mayi_hello_v6',
 'pudao_tengxun_external_dp_credit_score_v1',
 'pudao_tengxun_external_dp_loan_score_v1',
 'xys_fc92',
 'zj_50002']

In [23]:
fes_name_df = {
    'blz_yaosi_ty_2':'避雷针_耀思通用分2',
    'dxm_V12PRO-hjgdj2':'小满分V12PRO-互金版高定价2',
    'feat_loan_risk_age_from_idcard':'年龄',
    'feat_loan_risk_gender_from_idcard':'性别',
    'feat_loan_risk_province_from_idcard':'省份',
    'mayi_afc_yh_cus_dq_v2':'蚂蚁贷超分',
    'mayi_hello_v6':'蚂蚁定制分V6',
    'xys_fc92':'信易搜_翡翠分92',
    'xys_linglong92':'信易搜_玲珑分92',
    'zj_50002':'字节50002',
    'bairong_yunlianfen':'融安云联分-标准款',
    'pudao_tengxun_external_dp_loan_score_v1':'腾讯提现大盘定制分',
    'pudao_tengxun_external_dp_credit_score_v1':'腾讯授信大盘定制分',
}

In [24]:
#入模特征汇总信息
left_cats=[]
left_feas_infos = pd.DataFrame()
left_feas_infos['特征名'] = model_fes
left_feas_infos['中文注释'] = left_feas_infos['特征名'].map(fes_name_df)
left_feas_infos['特征类型'] = left_feas_infos.apply(lambda x: 'category' if x['特征名'] in cat_fes else 'float', axis=1)
left_feas_infos['重要性'] = model.feature_importance(importance_type='gain')
left_feas_infos['IV'] = left_feas_infos['特征名'].map(iv_res)
left_feas_infos = left_feas_infos.sort_values(by='重要性', ascending=False).reset_index(drop=True)
left_feas_infos['train_oot_psi'] = left_feas_infos['特征名'].map(psi_df['to'])
left_feas_infos['train缺失率'] = left_feas_infos['特征名'].map(train_lossrate)
left_feas_infos['oot缺失率'] = left_feas_infos['特征名'].map(oot_lossrate)
left_feas_infos

,特征名,中文注释,特征类型,重要性,IV,train_oot_psi,train缺失率,oot缺失率
0,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,float,31157.110579,0.135391,0.004129,0.000012,0.000000
1,mayi_hello_v6,蚂蚁定制分V6,float,14827.860605,0.110693,0.005675,0.000290,0.000042
2,mayi_afc_yh_cus_dq_v2,蚂蚁贷超分,float,13669.325775,0.124231,0.017076,0.000000,0.000000
3,zj_50002,字节50002,float,12082.647331,0.098862,0.024481,0.000198,0.000000
4,xys_fc92,信易搜_翡翠分92,float,8786.213327,0.121312,0.009828,0.000037,0.000000
5,blz_yaosi_ty_2,避雷针_耀思通用分2,float,7133.444443,0.128127,0.001382,0.000019,0.000000
6,feat_loan_risk_gender_from_idcard,性别,float,3534.518135,0.021534,0.003393,0.001075,0.002894
7,feat_loan_risk_age_from_idcard,年龄,float,3055.870215,0.013580,0.002023,0.000000,0.000000
8,pudao_tengxun_external_dp_loan_score_v1,腾讯提现大盘定制分,float,2234.062387,0.090966,0.014135,0.000099,0.000000
9,feat_loan_risk_province_from_idcard,省份,category,2187.923589,0.021978,0.001329,0.000043,0.000000


In [25]:
#入模特征分箱明细
fea_bins_df = pd.DataFrame()
for fea in left_feas_infos['特征名']:
    bins_one = iv_bins[fea]
    bins_one['特征名'] = fea
    fea_bins_df = pd.concat([fea_bins_df, bins_one])
    
fea_bins_df['Bin'] = fea_bins_df['Bin'] .astype('str')
fea_bins_df['中文注释'] = fea_bins_df['特征名'].map(fes_name_df)
fea_bins_df = fea_bins_df[['特征名', '中文注释', 'Bin', 'Count', 'Count (%)', 'Non-event', 'Event', 'Event rate', 'WoE', 'IV']]
fea_bins_df = fea_bins_df[fea_bins_df['Bin']!='Special']
fea_bins_df 

,特征名,中文注释,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV
0,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"(-inf, 450.500000)",20553,0.127028,19556,997,0.048509,-0.447586,0.031535
1,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[450.500000, 499.500000)",66813,0.412938,64336,2477,0.037074,-0.166801,0.012432
2,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[499.500000, 525.500000)",32906,0.203376,31981,925,0.028110,0.119231,0.002735
3,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[525.500000, 565.500000)",28496,0.176120,27938,558,0.019582,0.489512,0.033783
4,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[565.500000, inf)",13029,0.080526,12880,149,0.011436,1.035613,0.054906
...,...,...,...,...,...,...,...,...,...,...
2,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[6.385950, 7.658250)",45247,0.279649,43843,1404,0.031030,0.017418,0.000084
3,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[7.658250, 10.218950)",62012,0.383266,59671,2341,0.037751,-0.185604,0.014415
4,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[10.218950, inf)",8173,0.050513,7792,381,0.046617,-0.405819,0.010101
6,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,Missing,6,0.000037,6,0,0.000000,0.0,0.000000


In [26]:
### OOT样本分箱
from optbinning import OptimalBinning
df_merge_oot = df_oot.copy()

fea_bins_df_oot = pd.DataFrame()
for fes_one in left_feas_infos['特征名']:
    # splits_tmp = [-np.inf] + fes_optb[fes_one].splits.tolist() + [np.inf]
    df_merge_oot['Bin'] = fes_optb[fes_one].transform(df_merge_oot[fes_one], metric='bins',
                                           metric_special='empirical',
                                           metric_missing='empirical',
                                           show_digits = 6
                                           )
    df_tmp_normal = df_merge_oot[(~df_merge_oot[fes_one].isin(special_list))&(df_merge_oot[fes_one].notnull())]
    df_tmp_special = df_merge_oot[df_merge_oot[fes_one].isin(special_list)]
    df_tmp_null = df_merge_oot[df_merge_oot[fes_one].isnull()]
    all_num_tmp = df_merge_oot.shape[0]
    if df_tmp_normal.shape[0]>0:
        stat_tmp_normal = df_tmp_normal.groupby(['Bin']).agg({id_col:'count', y_label:'sum'})
        stat_tmp_normal = stat_tmp_normal.reset_index()
        stat_tmp_normal.columns = ['Bin', 'Count', 'Event']
        list_sort_tmp = fea_bins_df[(fea_bins_df['特征名']==fes_one) & (~fea_bins_df['Bin'].isin(['Missing', '']))]['Bin'].tolist()
        stat_tmp_normal['Bin'] = stat_tmp_normal['Bin'].astype('category').cat.set_categories(list_sort_tmp)
        stat_tmp_normal = stat_tmp_normal.sort_values(by='Bin')
        stat_tmp_normal['Bin'] = stat_tmp_normal['Bin'].astype('str')
        stat_tmp_normal.index = stat_tmp_normal.index.astype('str')
        # stat_tmp_normal['Count (%)'] = stat_tmp_normal['Count']/all_num_tmp
    else:
        stat_tmp_normal = pd.DataFrame()

    if df_tmp_special.shape[0]>0:
        stat_tmp_special = pd.DataFrame({'Bin':['Special'], 'Count':[df_tmp_special[y_label].count()], 'Event':[df_tmp_special[y_label].sum()]})
    else:
        stat_tmp_special = pd.DataFrame({'Bin':['Special'], 'Count':[0], 'Event':[0]})

    if df_tmp_null.shape[0]>0:
        stat_tmp_null = pd.DataFrame({'Bin':['Missing'], 'Count':[df_tmp_null[y_label].count()], 'Event':[df_tmp_null[y_label].sum()]})
    else:
        stat_tmp_null = pd.DataFrame({'Bin':['Missing'], 'Count':[0], 'Event':[0]})

    splits_oot_one = pd.concat([stat_tmp_normal, stat_tmp_special, stat_tmp_null])
    if fes_one in cat_fes:
        category_tmp = pd.CategoricalDtype(categories=fea_bins_df[fea_bins_df['特征名']==fes_one]['Bin'].tolist()[:-1], ordered=True)
        splits_oot_one['Bin'] = splits_oot_one['Bin'].astype(category_tmp)
        splits_oot_one = splits_oot_one.sort_values(by='Bin', ascending=True)
        splits_oot_one['Bin'] = splits_oot_one['Bin'].astype('str')

    if splits_oot_one.shape[0]>0:
        stat_tmp_total = pd.DataFrame({'Bin':['Totals'], 'Count':[splits_oot_one['Count'].sum()], 'Event':[splits_oot_one['Event'].sum()]})
        splits_oot_one = pd.concat([splits_oot_one, stat_tmp_total])
        splits_oot_one['Count (%)'] = splits_oot_one['Count']/all_num_tmp
        splits_oot_one['Non-event'] = splits_oot_one['Count'] - splits_oot_one['Event']
        splits_oot_one['Event rate'] = splits_oot_one['Event'] / splits_oot_one['Count']
        splits_oot_one['特征名'] = fes_one
        splits_oot_one['中文注释'] = splits_oot_one['特征名'].map(fes_name_df)
        splits_oot_one = splits_oot_one[['特征名', '中文注释', 'Bin', 'Count', 'Count (%)', 'Non-event', 'Event', 'Event rate']].fillna(0)

    fea_bins_df_oot = pd.concat([fea_bins_df_oot, splits_oot_one])
fea_bins_df_oot = fea_bins_df_oot[fea_bins_df_oot['Bin']!='Special']
fea_bins_df_oot


,特征名,中文注释,Bin,Count,Count (%),Non-event,Event,Event rate
0,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"(-inf, 450.500000)",3483,0.146081,3265.0,218.0,0.062590
1,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[450.500000, 499.500000)",9844,0.412868,9398.0,446.0,0.045307
2,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[499.500000, 525.500000)",4896,0.205343,4725.0,171.0,0.034926
3,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[525.500000, 565.500000)",4017,0.168477,3907.0,110.0,0.027384
4,dxm_V12PRO-hjgdj2,小满分V12PRO-互金版高定价2,"[565.500000, inf)",1603,0.067231,1575.0,28.0,0.017467
...,...,...,...,...,...,...,...,...
3,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[6.385950, 7.658250)",6473,0.271484,6226.0,247.0,0.038159
4,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[7.658250, 10.218950)",10282,0.431238,9792.0,490.0,0.047656
1,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,"[10.218950, inf)",1544,0.064757,1468.0,76.0,0.049223
0,pudao_tengxun_external_dp_credit_score_v1,腾讯授信大盘定制分,Missing,0,0.000000,0.0,0.0,0.000000


In [27]:
model_params = model.params
model_params['num_trees'] = model.num_trees()
model_infos = pd.DataFrame(pd.Series(model_params),columns=['参数取值'])
model_infos

,参数取值
boosting_type,gbdt
objective,binary
metrics,auc
learning_rate,0.07
max_depth,3
num_leaves,11
max_bin,11
subsample,0.7
colsample_bytree,1.0
lambda_l1,54


In [69]:
def Prob2Score(prob, basePoint=500, PDO=200):
    # ll = np.polyfit([153,1083],[301,899],1)
    ll = [0.6430107, 202.6193548]
    score = int(basePoint - PDO /np.log(2) * np.log(prob/(1 - prob)))
    score = int(score*ll[0]+ll[1])
    score = min(max([score,300]),900)
    return score

In [70]:
ll = np.polyfit([153,1083],[301,899],1)
print(ll[0], ll[1])

0.6430107526881719 202.61935483870943


In [71]:
train[model_score] = train[model_pred].apply(lambda x: Prob2Score(x,))
test[model_score] = test[model_pred].apply(lambda x: Prob2Score(x, ))
df_oot[model_score] = df_oot[model_pred].apply(lambda x: Prob2Score(x, ))
df_oot1[model_score] = df_oot1[model_pred].apply(lambda x: Prob2Score(x,))

df_oot1[model_score].min(),df_oot1[model_score].max(), train[model_score].min(),train[model_score].max(), test[model_score].min(),test[model_score].max(),

(300, 900, 300, 900, 300, 900)

In [77]:
model_score_list = [train, test, df_oot, df_oot1]
model_score_name = ['train', 'test', 'oot(去除灰)', 'oot(灰为好)',]

cut_ks_lift = {}
qcut_ks_lift = {}
pred_infos = pd.DataFrame()
for i, model_score_one in enumerate(model_score_list):
    model_score_report_one = mf.model_score_report(model_score_one, model_score, y_label, auc_score=model_pred, max_bins=10, precision=0, sort=if_sort, self_bins=self_cut_dict['10分箱'],
                                                   lowerlift_per=[0.01, 0.02, 0.03, 0.04])
    pred_infos[model_score_name[i]] = pd.concat([pd.Series({'AUC':model_score_report_one['AUC'], 
                                                            'KS':model_score_report_one['KS'],}), 
                                                 model_score_report_one['TH_lift'],
                                                 pd.Series({'尾部1%lift':model_score_report_one['lower_lift']['小于0.01lift'], 
                                                            '尾部2%lift':model_score_report_one['lower_lift']['小于0.02lift'], 
                                                            '尾部3%lift':model_score_report_one['lower_lift']['小于0.03lift'], 
                                                            '尾部4%lift':model_score_report_one['lower_lift']['小于0.04lift'], 
                                                            }), ])
    qcut_ks_lift[model_score_name[i]] = model_score_report_one['qcut_ks_lift']
    cut_ks_lift[model_score_name[i]] = model_score_report_one['cut_ks_lift']

pred_infos

,train,test,oot(去除灰),oot(灰为好)
AUC,0.666546,0.662848,0.659999,0.651054
KS,0.241093,0.246418,0.256370,0.243851
尾部5%lift,2.476826,2.459350,2.428019,2.304656
尾部10%lift,2.208110,2.157106,2.045461,1.971436
头部10%lift,0.249329,0.375695,0.267136,0.296279
头部5%lift,0.204838,0.452734,0.266355,0.267157
尾部5%EventRate,0.078163,0.077641,0.099084,0.080622
尾部10%EventRate,0.069683,0.068099,0.083472,0.068966
头部5%EventRate,0.006464,0.014293,0.010870,0.009346
头部10%EventRate,0.007868,0.011861,0.010901,0.010365


In [78]:
cut_ks_lift['oot(灰为好)']

,Event,NEvent,all,EventRate,Event_CumRate,NEvent_CumRate,all_CumRate,all_Rate,ks,lift,cum_lift
"(-inf, 360.0]",60.0,624.0,684.0,0.087719,0.061665,0.023248,0.024592,0.024592,0.038417,2.507528,2.507528
"(360.0, 420.0]",174.0,2646.0,2820.0,0.061702,0.240493,0.121829,0.125980,0.101388,0.118665,1.763806,1.908984
"(420.0, 480.0]",280.0,5383.0,5663.0,0.049444,0.528263,0.322380,0.329582,0.203603,0.205883,1.413390,1.602826
"(480.0, 540.0]",235.0,6159.0,6394.0,0.036753,0.769784,0.551842,0.559466,0.229884,0.217942,1.050620,1.375926
"(540.0, 600.0]",126.0,4662.0,4788.0,0.026316,0.899281,0.725532,0.731610,0.172144,0.173749,0.752258,1.229180
"(600.0, 660.0]",46.0,3249.0,3295.0,0.013961,0.946557,0.846578,0.850076,0.118466,0.099979,0.399074,1.113498
"(660.0, 720.0]",29.0,1943.0,1972.0,0.014706,0.976362,0.918967,0.920975,0.070900,0.057395,0.420380,1.060139
"(720.0, 780.0]",12.0,1082.0,1094.0,0.010969,0.988695,0.959279,0.960308,0.039333,0.029416,0.313556,1.029560
"(780.0, 840.0]",6.0,558.0,564.0,0.010638,0.994861,0.980068,0.980585,0.020278,0.014793,0.304104,1.014559
"(840.0, inf]",5.0,535.0,540.0,0.009259,1.000000,1.000000,1.000000,0.019415,0.000000,0.264683,1.000000


### 分客群分析

In [79]:
df_merge_tmp1 = df_merge.copy()
df_merge_tmp1[label_list] = df_merge_tmp1[label_list].replace(0.5, np.nan)
df_dp_qh = df_merge_tmp1.copy()
df_zy_qh = df_merge_tmp1[df_merge_tmp1.is_zy==1]
df_fzy_qh = df_merge_tmp1[df_merge_tmp1.is_zy!=1]
df_zy_api_qh = df_merge_tmp1[(df_merge_tmp1.is_zy==1) & (df_merge_tmp1.channel_type=='api')]
df_zy_napi_qh = df_merge_tmp1[(df_merge_tmp1.is_zy==1) & (df_merge_tmp1.channel_type=='app')]



df_merge_tmp2 = df_merge.copy()
df_merge_tmp2[label_list] = df_merge_tmp2[label_list].replace(0.5, 0)
df_dp = df_merge_tmp2.copy()
df_zy = df_merge_tmp2[df_merge_tmp2.is_zy==1]
df_fzy = df_merge_tmp2[df_merge_tmp2.is_zy!=1]
df_zy_api = df_merge_tmp2[(df_merge_tmp2.is_zy==1) & (df_merge_tmp2.channel_type=='api')]
df_zy_napi = df_merge_tmp2[(df_merge_tmp2.is_zy==1) & (df_merge_tmp2.channel_type=='app')]

df_dp.shape, df_zy.shape, df_fzy.shape, df_zy_api.shape, df_zy_napi.shape

((300000, 71), (211235, 71), (88765, 71), (153231, 71), (57992, 71))

In [80]:
kq_list = {'大盘(去灰)':df_dp_qh, '自营(去灰)':df_zy_qh, '非自营(去灰)':df_fzy_qh, '自营API(去灰)':df_zy_api_qh, '自营非API(去灰)':df_zy_napi_qh,
           '大盘(灰为好)':df_dp, '自营(灰为好)':df_zy, '非自营(灰为好)':df_fzy, '自营API(灰为好)':df_zy_api, '自营非API(灰为好)':df_zy_napi, }
kq_tmp = [ '大盘(去灰)', '自营(去灰)', '非自营(去灰)', '自营API(去灰)', '自营非API(去灰)', '大盘(灰为好)', '自营(灰为好)', '非自营(灰为好)', '自营API(灰为好)', '自营非API(灰为好)']

In [81]:
##样本情况

mon_sample_info_all = {}
for kq_one in kq_tmp:
    model_df = kq_list[kq_one]
    mon_sample_info_one = {}
    #各个月份样本情况
    sampple_br = model_df.groupby(parts_col)[label_list].mean()
    sampple_br = pd.concat([sampple_br, pd.DataFrame(model_df[label_list].mean(), columns=['合计']).T])
    # sampple_br = sampple_br.append(pd.DataFrame(model_df[label_list].mean(), columns=['合计']).T)
    sampple_bnum = model_df.groupby(parts_col)[label_list].sum()
    sampple_bnum = pd.concat([sampple_bnum, pd.DataFrame(model_df[label_list].sum(), columns=['合计']).T])
    # sampple_bnum = sampple_bnum.append(pd.DataFrame(model_df[label_list].sum(), columns=['合计']).T)

    all_score_df_copy = model_df.copy()
    all_score_df_copy[label_list] = all_score_df_copy[label_list].notnull()
    sampple_num = all_score_df_copy.groupby(parts_col)[label_list].sum()
    sampple_num = pd.concat([sampple_num, pd.DataFrame(all_score_df_copy[label_list].sum(), columns=['合计']).T])
    # sampple_num = sampple_num.append(pd.DataFrame(all_score_df_copy[label_list].sum(), columns=['合计']).T)

    mon_sample_info_one['样本坏率'] = sampple_br
    mon_sample_info_one['样本坏量'] = sampple_bnum
    mon_sample_info_one['样本总量'] = sampple_num
    mon_sample_info_all[kq_one] = mon_sample_info_one

In [82]:
## PSI稳定性
psi_all = {} 
for score_one in tqdm(model_scores):
    psi_score_one = {}
    for kq_one in kq_tmp:
        score_kq_one = kq_list[kq_one]
        model_df = score_kq_one.copy()
        psi_pred_df = model_df[[date_col, parts_col, score_one]]
        mons = np.sort(psi_pred_df[parts_col].unique())
        psi_res = {}
        psi_base = {}
        psi_df_model = pd.DataFrame()
        base_score = psi_pred_df[psi_pred_df[parts_col].isin(['2024-10', '2024-11', '2024-12'])][score_one]
        for i, mon in enumerate(mons):
            base_df = psi_pred_df[psi_pred_df[parts_col]==mon][score_one]
            psi_base[mon], _, _ = mf.calculate_psi(base_score, base_df, bins=10, min_sample=10, numerical=True)
            if mon<max(mons):
                mon_n = mons[i+1]
                test_df = psi_pred_df[psi_pred_df[parts_col]==mon_n][score_one]
                psi_res[mon_n], _ , _= mf.calculate_psi(base_df, test_df, bins=10, min_sample=10, numerical=True)
        psi_df_model['2024为基期'] = pd.Series(psi_base)
        psi_df_model['月环比'] = pd.Series(psi_res)
        psi_score_one[kq_one] = psi_df_model
    psi_all[score_one] = psi_score_one


100%|██████████| 1/1 [00:02<00:00,  2.58s/it]


In [83]:
psi_all['model_score']['大盘(灰为好)']

,2024为基期,月环比
2024-10,0.000199,NaN
2024-11,0.001602,0.002053
2024-12,0.001359,0.005594
2025-01,0.004954,0.001439
2025-02,0.019322,0.005443
2025-03,0.039583,0.003588
2025-04,0.009130,0.022475
2025-05,0.039827,0.014927


In [84]:
##分数分布
mon_dis_all = {}
for score_one in tqdm(model_scores):
    mon_dis_score_one = {}
    for kq_one in kq_tmp:
        score_kq_one = kq_list[kq_one]
        model_df = score_kq_one.copy()
        psi_pred_df = model_df[[date_col, parts_col, score_one]]
        psi_pred_df['score_cut'] = pd.cut(psi_pred_df[model_score], bins=self_cut_dict['10分箱'])
        mon_dis_stat = psi_pred_df.groupby(['score_cut', parts_col])[model_score].count()
        mon_dis_stat = mon_dis_stat.unstack()
        mon_dis_stat = mon_dis_stat/mon_dis_stat.sum()
        cols_tmp = mon_dis_stat.columns.tolist()
        index_tmp = mon_dis_stat.index.astype('str').tolist()
        mon_dis_stat.columns = cols_tmp
        mon_dis_stat.index = index_tmp
        mon_dis_score_one[kq_one] = mon_dis_stat
    mon_dis_all[score_one] = mon_dis_score_one

100%|██████████| 1/1 [00:00<00:00,  1.99it/s]


In [85]:
mon_dis_all['model_score']['大盘(灰为好)']

,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05
"(-inf, 360.0]",0.016259,0.015763,0.019273,0.020298,0.021358,0.023884,0.021077,0.025297
"(360.0, 420.0]",0.075869,0.071005,0.079242,0.082871,0.092138,0.101514,0.086740,0.109022
"(420.0, 480.0]",0.168654,0.158463,0.174200,0.181358,0.192746,0.202883,0.184619,0.203492
"(480.0, 540.0]",0.209741,0.210364,0.212721,0.215638,0.224438,0.228718,0.211076,0.218267
"(540.0, 600.0]",0.187129,0.190846,0.188175,0.185224,0.182391,0.176326,0.172556,0.160734
"(600.0, 660.0]",0.140298,0.144908,0.136473,0.129763,0.123789,0.116491,0.124267,0.113947
"(660.0, 720.0]",0.092153,0.095354,0.087586,0.083401,0.076211,0.072337,0.084753,0.072980
"(720.0, 780.0]",0.053159,0.055103,0.051088,0.051022,0.043060,0.038698,0.053497,0.050369
"(780.0, 840.0]",0.026677,0.027579,0.024469,0.024694,0.021398,0.019421,0.028068,0.025968
"(840.0, inf]",0.030060,0.030616,0.026772,0.025732,0.022472,0.019728,0.033346,0.019924


In [86]:
##分月性能评估
mon_info_all = {}
for score_one in tqdm(model_scores):
    # model_pred = score_one
    # model_score = score_one

    mon_info_score_one = {}

    for kq_one in kq_tmp:
        mon_info_kq_one = {}
        score_kq_one = kq_list[kq_one]
        #所有y标测算
        need_cols = list(set([parts_col, model_pred, model_score]+label_list))
        all_y_name = label_list.copy()
        all_score_df = score_kq_one[need_cols]

        parts = np.sort(all_score_df[parts_col].unique())
        part_name = np.sort(all_score_df[parts_col].unique())


        ks_all_df = pd.DataFrame()
        t5p_lift_all = pd.DataFrame()
        t10p_lift_all = pd.DataFrame()
        h10p_lift_all = pd.DataFrame()
        h5p_lift_all = pd.DataFrame()
        t2p_lift_all = pd.DataFrame()
        qcut_ks_lift_all = {}
        for i, part in enumerate(parts):
            ks_part = {}
            t5p_lift_part = {}
            t10p_lift_part = {}
            h10p_lift_part = {}
            h5p_lift_part = {}
            t2p_lift_part = {}
            qcut_ks_lift_part = {}
            for y_one in all_y_name:
                try:
                    df_temp = all_score_df[(all_score_df[parts_col].isin([part])) & (all_score_df[y_one].notnull())]
                    model_score_report_one = mf.model_score_report(df_temp, model_score, y_one, auc_score=model_pred, max_bins=10, precision=0, sort=if_sort, self_bins=self_cut_dict['10分箱'],
                                                                   lowerlift_per=[0.02])
                    ks_part[y_one] = model_score_report_one['KS']
                    t5p_lift_part[y_one] = model_score_report_one['TH_lift']['尾部5%lift']
                    t10p_lift_part[y_one] = model_score_report_one['TH_lift']['尾部10%lift']
                    h10p_lift_part[y_one] = model_score_report_one['TH_lift']['头部10%lift']
                    h5p_lift_part[y_one] = model_score_report_one['TH_lift']['头部5%lift']
                    t2p_lift_part[y_one] = model_score_report_one['lower_lift']['小于0.02lift']
                    qcut_ks_lift_part[y_one] = model_score_report_one['qcut_ks_lift']
                except:
                    pass
            
            ks_all_df[part_name[i]] = pd.Series(ks_part)
            t5p_lift_all[part_name[i]] = pd.Series(t5p_lift_part)
            t10p_lift_all[part_name[i]] = pd.Series(t10p_lift_part)
            h10p_lift_all[part_name[i]] = pd.Series(h10p_lift_part)
            h5p_lift_all[part_name[i]] = pd.Series(h5p_lift_part)
            t2p_lift_all[part_name[i]] = pd.Series(t2p_lift_part)
            qcut_ks_lift_all[part_name[i]] = qcut_ks_lift_part
        mon_info_kq_one['KS'] = ks_all_df.T
        mon_info_kq_one['尾部5%lift'] = t5p_lift_all.T
        mon_info_kq_one['尾部10%lift'] = t10p_lift_all.T
        mon_info_kq_one['头部10%lift'] = h10p_lift_all.T
        mon_info_kq_one['头部5%lift'] = h5p_lift_all.T
        mon_info_kq_one['尾部2%lift'] = t2p_lift_all.T
        mon_info_score_one[kq_one] = mon_info_kq_one
    mon_info_all[score_one] = mon_info_score_one

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:15<00:00, 15.02s/it]


In [87]:
mon_info_all[model_score]['大盘(灰为好)']['KS']

,label_fpd_7,label_fpd_15,label_fpd_30,label_2pd_7,label_2pd_15,label_2pd_30,label_3pd_7,label_3pd_15,label_3pd_30,label_4pd_15,label_4pd_30,label_5pd_30,label_6pd_30
2024-10,0.223630,0.218199,0.231543,0.232524,0.235130,0.245393,0.216621,0.220833,0.222128,0.202084,0.201690,0.195252,0.174175
2024-11,0.212608,0.209961,0.224854,0.217930,0.221475,0.220855,0.209732,0.212492,0.214712,0.208426,0.207890,0.191420,0.191422
2024-12,0.231662,0.247706,0.259716,0.238313,0.248002,0.238714,0.222971,0.224894,0.223518,0.207653,0.209658,0.209121,NaN
2025-01,0.246096,0.245063,0.260413,0.237120,0.241476,0.241865,0.228266,0.232521,0.234219,0.200587,0.220890,NaN,NaN
2025-02,0.230280,0.229727,0.226380,0.224520,0.215203,0.220356,0.217762,0.214463,0.227218,0.224217,NaN,NaN,NaN
2025-03,0.256894,0.258296,0.270765,0.237970,0.246196,0.252657,0.226041,0.293925,NaN,NaN,NaN,NaN,NaN
2025-04,0.238523,0.240405,0.287460,0.299888,0.423759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05,0.214167,0.159737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
#OOT分箱

oot_bins_all = {}
for score_one in tqdm(model_scores):
    # model_pred = score_one
    # model_score = score_one
    score_one_oot_bins_tmp = {}
    for kq_one in kq_tmp:
        score_kq_one = kq_list[kq_one]
        model_df = score_kq_one.copy()

        m9_report_bins = pd.DataFrame()

        model_df_m9 = model_df[model_df['date']>oot_date]
        bins_dict = {'10分箱':10, '20分箱':20, '40分箱':40, '50分箱':50,}
        for y_one in label_list:
            df_temp = model_df_m9[(model_df_m9[y_one].notnull())]
           
            
            for bins_key in bins_dict.keys():
                try:
                    model_score_report_one = mf.model_score_report(df_temp, model_score, y_one, auc_score=model_pred, max_bins=bins_dict[bins_key], precision=0, sort=if_sort, self_bins=self_cut_dict[bins_key])
                    
                    qcut_report_one = model_score_report_one['qcut_ks_lift']
                    cut_report_one = model_score_report_one['cut_ks_lift']
                    qcut_report_one = qcut_report_one.reset_index().rename(columns={'index':'bins'})
                    cut_report_one = cut_report_one.reset_index().rename(columns={'index':'bins'})
                    
                    qcut_report_one.insert(0, '分箱数', bins_key)
                    cut_report_one.insert(0, '分箱数', bins_key)
                    
                    qcut_report_one.insert(0, 'y标', y_one)
                    cut_report_one.insert(0, 'y标', y_one)
                    
                    qcut_report_one.insert(0, '等频/等距', '等频')
                    cut_report_one.insert(0, '等频/等距', '等距')
                    
                    report_one_bins = pd.concat([qcut_report_one, cut_report_one])
                    m9_report_bins = pd.concat([m9_report_bins, report_one_bins])
                except:
                    pass
        
        score_one_oot_bins_tmp[kq_one] = m9_report_bins
    oot_bins_all[score_one] = score_one_oot_bins_tmp
                    

100%|██████████| 1/1 [00:07<00:00,  7.36s/it]


In [89]:
#月度分箱
mon_bins_all = {}
for score_one in tqdm(model_scores):
    # model_pred = score_one
    # model_score = score_one
    score_one_mon_bins_tmp = {}
    for kq_one in kq_tmp:
        score_kq_one = kq_list[kq_one]
        model_df = score_kq_one.copy()

        ##分月分箱明细
        mon_report_bins = pd.DataFrame()

        need_cols = list(set([parts_col, model_score, model_pred]+label_list))
        all_score_df = model_df[need_cols]

        parts = np.sort(all_score_df[parts_col].unique())
        part_name = np.sort(all_score_df[parts_col].unique())


        bins_dict = {'10分箱':10, '20分箱':20, '40分箱':40, '50分箱':50,}
        for i, part in enumerate(parts):
            part_name_one = part_name[i]
            for y_one in label_list:
                df_temp = all_score_df[(all_score_df[y_one].notnull())&(all_score_df[parts_col].isin([part]))]
                
                for bins_key in bins_dict.keys():
                    try:
                        model_score_report_one = mf.model_score_report(df_temp, model_score, y_one, auc_score=model_pred, max_bins=bins_dict[bins_key], precision=0, sort=if_sort, self_bins=self_cut_dict[bins_key])
                        # m9_qcut_y_one[bins_key] = model_score_report_one['qcut_ks_lift']
                        # m9_cut_y_one[bins_key] = model_score_report_one['cut_ks_lift']
                        
                        qcut_report_one = model_score_report_one['qcut_ks_lift']
                        cut_report_one = model_score_report_one['cut_ks_lift']
                        qcut_report_one = qcut_report_one.reset_index().rename(columns={'index':'bins'})
                        cut_report_one = cut_report_one.reset_index().rename(columns={'index':'bins'})
                        
                        qcut_report_one.insert(0, '分箱数', bins_key)
                        cut_report_one.insert(0, '分箱数', bins_key)
                        
                        qcut_report_one.insert(0, 'y标', y_one)
                        cut_report_one.insert(0, 'y标', y_one)
                        
                        qcut_report_one.insert(0, '等频/等距', '等频')
                        cut_report_one.insert(0, '等频/等距', '等距')

                        qcut_report_one.insert(0, '月份', part_name_one)
                        cut_report_one.insert(0, '月份', part_name_one)
                        
                        report_one_bins = pd.concat([qcut_report_one, cut_report_one])
                        # mon_report_bins = pd.concat([mon_report_bins, qcut_report_one])
                        mon_report_bins = pd.concat([mon_report_bins, report_one_bins])
                    except:
                        pass
            
        # mon_report_bins['bins'] = mon_report_bins['bins'].astype('str').str.replace('(-inf', '[300')
        # mon_report_bins['bins'] = mon_report_bins['bins'].astype('str').str.replace('inf]', '900]')
        score_one_mon_bins_tmp[kq_one] = mon_report_bins
    mon_bins_all[score_one] = score_one_mon_bins_tmp

100%|██████████| 1/1 [00:53<00:00, 53.59s/it]


In [90]:
df_corr = train[model_fes].corr('pearson')

In [91]:
writer = pd.ExcelWriter('model_report_tmp.xlsx')
sample_info_name_list = ['样本坏率', '样本坏量', '样本总量']
xn_name_list = ['KS', '尾部5%lift', '尾部10%lift', '头部10%lift', '头部5%lift', '尾部2%lift']

a = 11
b = 14
c = 150

for i, kq_one in enumerate(kq_tmp):
      pd.Series(kq_one).to_excel(writer, sheet_name='分月样本情况', startrow=0, startcol=i*15, na_rep='/', index=False, header=False)
      for j, name_one in enumerate(sample_info_name_list):
            mon_sample_info_all[kq_one][name_one].to_excel(writer, sheet_name='分月样本情况', startrow=j*a+1, startcol=i*15, na_rep='/')
            pd.Series(name_one).to_excel(writer, sheet_name='分月样本情况', startrow=j*a+1, startcol=i*15, na_rep='/', index=False, header=False)

samples_info.to_excel(writer, sheet_name='建模概述', startrow=0, startcol=0, na_rep='/')
pd.Series('建模样本').to_excel(writer, sheet_name='建模概述', startrow=0, startcol=0, na_rep='/', index=False, header=False)
model_infos.to_excel(writer, sheet_name='建模概述', startrow=10, startcol=0, na_rep='/')
pd.Series('模型参数').to_excel(writer, sheet_name='建模概述', startrow=10, startcol=0, na_rep='/', index=False, header=False)
pred_infos.to_excel(writer, sheet_name='建模概述', startrow=40, startcol=0, na_rep='/')
pd.Series('模型性能').to_excel(writer, sheet_name='建模概述', startrow=40, startcol=0, na_rep='/', index=False, header=False)

left_feas_infos.to_excel(writer, sheet_name='入模特征', startrow=0, startcol=0, na_rep='/')
fea_bins_df.to_excel(writer, sheet_name='入模特征', startrow=c, startcol=0, na_rep='/')
fea_bins_df_oot.to_excel(writer, sheet_name='入模特征', startrow=c, startcol=20, na_rep='/')

pd.Series('等频分箱').to_excel(writer, sheet_name='建模分箱明细', startrow=0, startcol=0, na_rep='/', index=False, header=False)
pd.Series('等距分箱').to_excel(writer, sheet_name='建模分箱明细', startrow=0, startcol=14, na_rep='/', index=False, header=False)
for i, model_score in enumerate(model_score_name):
    qcut_ks_lift_one = qcut_ks_lift[model_score]
    cut_ks_lift_one = cut_ks_lift[model_score]
    qcut_ks_lift_one.to_excel(writer, sheet_name='建模分箱明细', startrow=i*b+1, startcol=0, na_rep='/')
    pd.Series(model_score).to_excel(writer, sheet_name='建模分箱明细', startrow=i*b+1, startcol=0, na_rep='/', index=False, header=False)
    cut_ks_lift_one.to_excel(writer, sheet_name='建模分箱明细', startrow=i*b+1, startcol=14, na_rep='/')
    pd.Series(model_score).to_excel(writer, sheet_name='建模分箱明细', startrow=i*b+1, startcol=14, na_rep='/', index=False, header=False)

        
df_corr.to_excel(writer, sheet_name='入模特征相关系数', startrow=0, startcol=0, na_rep='/')

for score_one in model_scores:
      for i, kq_one in enumerate(kq_tmp):
            pd.Series(kq_one).to_excel(writer, sheet_name=score_one, startrow=0, startcol=i*15, na_rep='/', index=False, header=False)
            psi_all[score_one][kq_one].to_excel(writer, sheet_name=score_one, startrow=1, startcol=i*15, na_rep='/')
            pd.Series('PSI').to_excel(writer, sheet_name=score_one, startrow=1, startcol=i*15, na_rep='/', index=False, header=False)
            mon_dis_all[score_one][kq_one].to_excel(writer, sheet_name=score_one, startrow=16, startcol=i*15, na_rep='/')
            pd.Series('样本分布').to_excel(writer, sheet_name=score_one, startrow=16, startcol=i*15, na_rep='/', index=False, header=False)

            oot_bins_all[score_one][kq_one].to_excel(writer, sheet_name='oot_'+score_one+'_'+kq_one, startrow=1, startcol=0, na_rep='/', index=False)
            pd.Series('OOT样本').to_excel(writer, sheet_name='oot_'+score_one+'_'+kq_one, startrow=0, startcol=0, na_rep='/', index=False, header=False)
            mon_bins_all[score_one][kq_one].to_excel(writer, sheet_name='mon_'+score_one+'_'+kq_one, startrow=0, startcol=0, na_rep='/')


            
            for k, xn_name_one in enumerate(xn_name_list):
                  mon_info_all[score_one][kq_one][xn_name_one].to_excel(writer, sheet_name=score_one, startrow=28+k*a, startcol=i*15, na_rep='/')
                  pd.Series(xn_name_one).to_excel(writer, sheet_name=score_one, startrow=28+k*a, startcol=i*15, na_rep='/', index=False, header=False)
      

writer.close()